這一章會討論一些有用的應用程式知識和一些新的主題  
* 如何組織物件
* 再一次討論資料與行為
* 以屬性將資料包裝在行為中
* 以行為限制資料
* 不重複原則
* 辨識重複程式

# 視物件為物件
---
辨識出物件非常重要  
物件是有資料與行為的一個東西  
  
如果我們只操控資料，通常以清單、組、字典或其他資料結構儲存  
如果只有行為但不存資料，簡單的函式更為適合  
  
隨著程式擴展  
才考慮將變數和函式組成類別物件  

In [ ]:
square = [(1,1), (1,2), (2,2), (2,1)]

import math

def distance(p1,p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def perimeter(polygon):
    perimeter = 0
    points = polygon + [polygon[0]]
    for i in range(len(polygon)):
        perimeter += distance(points[i],points[i+1])
    return perimeter

若以物件導向去改寫上述例子  
將會分成 Point 與 Polygon 兩個類別  
Point儲存座標與計算兩點距離  
Polygon儲存多個座標與計算周長

In [ ]:
import math

class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

    def distance(self, p2):
        return math.sqrt((self.x - p2.x)**2 + (self.y - p2.y)**2)

class Polygon:
    def __init__(self) -> None:
        self.vertices = []
    
    def add_point(self, point):
        self.vertices.append(point)
    
    def perimeter(self):
        perimeter = 0
        points = self.vertices + [self.vertices[0]]
        for i in range(len(self.vertices)):
            perimeter += points[i].distance(points[i+1])
        return perimeter

這版程式改寫後  
比原本的要多出兩倍長度  
但在使用上或是程式碼的解讀上  
改寫後的程式可讀性比起改寫前清楚很多  
也不用動用到 雙維度的 List  
  
程式長度不是程式複雜性的好指標  
一行程式做出複雜的巨量工作  
或許是有趣的練習  
但程式碼卻難以閱讀  
說不定過幾天後就看不懂了  
不要盲目地減少程式碼  

> 不要因為會使用物件就急著使用物件  
> **但千萬別在需要物件時省略物件**

# 對物件資料加上行為屬性
---
變數名稱前面加一個底線代表此變數是私用的(單底線Python看起來沒有強制，雙底線才有，但還是有方法可以不透過額外撰寫def呼叫出來)  
然後額外提供方法來存取變數  

In [ ]:
class Color:
    def __init__(self, rgb_value, name) -> None:
        self._rgb_value = rgb_value
        self._name = name
    
    def set_name(self, name):
        self._name = name
    
    def get_name(self):
        return self._name

c = Color('#ff0000', "bright red")
print(c._name)
print(c.get_name())
c.set_name("red")
print(c.get_name())

與直接存取的Python風格差很多

In [ ]:
class Color_Py:
    def __init__(self, rgb_value, name) -> None:
        self.rgb_value = rgb_value
        self.name = name

c = Color_Py(0xff0000, "bright red")
print(c.name)
c.name = "red"
print(c.name)

為何程式設計師會想要前述的寫法呢?    
或許是因為在程式設計中  
想設計一個「得到」和「設置」的按鈕來對應方法吧  
  
接下來我們將 Color 物件修改一下  
避免使用者傳入空值
接著增加 property 關鍵字  
讓方法看起來像是個「屬性」

In [ ]:
class Color_V:
    def __init__(self, rgb_value: int, name: str) -> None:
        self._rgb_value = rgb_value
        if not name:
            raise ValueError(f"Invalid name {name!r}")
        self._name = name

    def _set_name(self, name: str) -> None:
        if not name:
            raise ValueError(f"Invalid name {name!r}")
        self._name = name
    
    def _get_name(self) -> str:
        return self._name
    
    name = property(_get_name, _set_name) #括弧內不能相反，會報錯

這樣子新版的 Color 物件  
就能像大家熟悉的Python使用方式

In [ ]:
c = Color_V(0xff0000, "bright red")
print(c.name)
c.name = "red"
print(c.name)
c.name = ""

# 屬性的細節
---
property建構元可接受額外兩個參數  
del 和 屬性的字串描述

In [ ]:
class NorwegianBlue:
    def __init__(self, name: str) -> None:
        self._name = name
        self._state: str

    def _get_state(self) -> str:
        print(f"Getting {self._name}'s State")
        return self._state

    def _set_state(self, state: str) -> None:
        print(f"Setting {self._name}'s State to {state!r}")
        self._state = state
        
    def _del_state(self) -> None:
        print(f"{self._name} is pushing up daisies!")
        del self._state

    silly = property(_get_state, _set_state, _del_state,"This is a silly property")

In [ ]:
p = NorwegianBlue("Polly")
p.silly = "Pining for the fjords"
p.silly


In [ ]:
del p.silly

這邊注意到方法都是 _get_state, _set_state...  
但在物件後面卻都是接 property 指定的 silly 去執行  
這也是為什麼 Python 裡有些東西不用加括號就能產生回饋  

# 修飾詞、裝飾器 - 建構屬性的另一個方法
---
利用 Python 的裝飾器  
可以讓整個程式變得更易讀  


In [ ]:
class NorwegianBlue_P:
    def __init__(self, name: str) -> None:
        self._name = name
        self._state: str
        
    @property
    def silly(self) -> str:
        print(f"Getting {self._name}'s State")
        return self._state

上方的程式我們看到方法從 _get_state 變成 silly  
並且在他上方加上一個 @property   

In [ ]:
class NorwegianBlue_P:
    def __init__(self, name: str, state) -> None:
        self._name = name
        self._state = state

    @property
    def silly(self) -> str:
        """This is a silly property"""
        print(f"Getting {self._name}'s State")
        return self._state
        

In [ ]:
p = NorwegianBlue_P("Polly", "Pining for the fjords")
p.silly

此方法變成了一個 get 函式  
我們繼續為此物件加上 set 和 del   
並且把 self._state 改為 : str  
讓 _state 用 setting 的方式帶進去

In [ ]:
class NorwegianBlue_P:
    def __init__(self, name: str) -> None:
        self._name = name
        self._state: str

    @property
    def silly(self) -> str:
        """This is a silly property"""
        print(f"Getting {self._name}'s State")
        return self._state

    @silly.setter
    def silly(self, state: str) -> None:
        print(f"Setting {self._name}'s State to {state!r}")
        self._state = state

    @silly.deleter
    def silly(self) -> None:
        print(f"{self._name} is pushing up daisies!")
        del self._state

注意到 setter 和 deleter 的用法  
方法名稱也全部都是 silly  
而裝飾器是 @方法.setter 和 @方法.deleter  

In [ ]:
p = NorwegianBlue_P("Polly")
p.silly = "Pining for the fjords"
p.silly

In [ ]:
del p.silly

# 決定何時使用屬性
---
Attribute、Method、Property 都是 Class 的特徵  
https://www.learncodewithmike.com/2020/01/python-attribute.html (這篇有詳細介紹)  
方法是個可以被呼叫的特徵，通常代表動作
確定不是動作後  
就決定是要設為標準資料 或 屬性(Attribute or Property)  

In [ ]:
from urllib.request import urlopen

class WebPage:
    def __init__(self, url) -> None:
        self.url = url
        self._content = None
    
    @property
    def content(self):
        if not self._content:
            print("Retrieving New Page...")
            with urlopen(self.url) as response:
                self._content = response.read()
        return self._content

In [ ]:
import time
webpage = WebPage("http://ccphillips.net")
now = time.time()
content1 = webpage.content


In [ ]:
time.time()-now

In [ ]:
now = time.time()
content2 = webpage.content
time.time()-now

In [ ]:
content1 == content2

建立一個類別 AverageList 繼承自 list  
為他加上計算平均值的方法  
我們可以用 @property 為方法建立屬性  
也可以另外建一個方法叫做 calculate_average  

In [ ]:
class AverageList(list):
    @property
    def average(self):
        return sum(self)/len(self)
    
    def calculate_average(self):
        return sum(self)/len(self)

In [ ]:
a = AverageList([1,2,3,4])
print(a.average)
print(a.calculate_average())

這兩個方法的差別就在於使用時需不需要加括號  

# 管理員物件
---
現在來設計一個高階物件：用來管理其他物件的物件，此物件將所有東西綁在一起  
管理物件的屬性傾向參考其他執行〝可見〞工作的物件  
這種物件的行為是在正確時間委任任務給其他物件，並在物件之間傳遞訊息  
  
我們撰寫一個找尋與替代儲存於壓縮ZIP檔案中的文字檔案作範例  
我們需要一個物件代表 ZIP 檔
一個物件代表文字檔案  
管理物件要確保以下事情發生  
1. 解開壓縮檔
2. 執行找尋與替代動作
3. 壓縮到新檔案中

In [ ]:
import sys
import shutil
import zipfile
from pathlib import Path

class ZipReplace:
    def __init__(self, filename, search_string, replace_string) -> None:
        self.filename = filename
        self.search_string = search_string
        self.replace_string = replace_string
        self.temp_directory = Path(f"unzipped-{filename}")

之後我們可以對這三個步驟建構〝管理員〞方法  
此管理員方法將責任委派給其他方法，並依序執行  
或  
乾脆不要建立這個方法
  
拆分成三個步驟的好處是
* 可讀性：每個步驟自成一個單元更好閱讀
* 可擴充性：容易擴充，不用複製其他較無相關的方法
* 分割：外部類別可以建構這個類別的實例並直接呼叫

In [8]:
import sys
import shutil
import zipfile
from pathlib import Path

class ZipReplace:
    def __init__(self, filename, search_string, replace_string) -> None:
        self.filename = filename
        self.search_string = search_string
        self.replace_string = replace_string
        self.temp_directory = Path(f"unzipped-{filename}")
    
    def zip_find_replace(self):
        '''管理員方法，依序執行下面方法'''
        self.unzip_files()
        self.find_replace()
        self.zip_files()
    
    def unzip_files(self):
        '''
        建立一個暫存資料夾
        將檔案解壓縮到裡面
        '''
        self.temp_directory.mkdir()
        with zipfile.ZipFile(self.filename) as zip:
            zip.extractall(str(self.temp_directory))
    

    def find_replace(self):
        '''
        到暫存資料夾中開啟每一個檔
        讀取文中的內文
        並把每一個 search_string 換成 replace_string
        寫回文件
        '''
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open('w') as file:
                file.write(contents)
        
    def zip_files(self):
        '''
        將暫存資料夾內的檔案寫進壓縮檔
        並刪除暫存資料夾
        '''
        with zipfile.ZipFile(self.filename, 'w') as file:
            for filename in self.temp_directory.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.temp_directory))

# if __name__ == "__main__":
#     ZipReplace(*sys.argv[1:4]).zip_find_replace()

In [9]:
!python zipsearch.py hello.zip hello hi

# 移除重複的程式
---
為何重複的程式碼不好 ?  
大部分都與可讀性與維護性有關  
只要有人必須閱讀或理解此程式，而且在遇到重複的區塊時  
就要去理解這段程式與剛剛那一段有何不同，有何相同，在什麼情況下要呼叫哪一段  
就算只有自己是唯一讀者  
經過長時間後  
看起來也會像是別人寫的程式  
  
在維護上更是一種折磨  
要是只有維護到一段  
另一段沒維護到  
很容易出現煩人的錯誤  
  
導致的結果就是  
閱讀和修改程式的人  
比一開始就規劃不重複寫還要花更多時間  

試著不要重複自己吧  
遵循 Python 優雅的原則

# 實務
---
剛剛撰寫了取代 zip 中的文件部分字串在壓縮的小程式  
現在我們收到了要將圖片放大成 640 X 480 的要求  
順便想讓程式也能開啟 tar 檔時  
我們該怎麼修改呢?

我們嘗試用繼承來改  
先將原有的 ZipReplace 修改成通用的超類別

In [3]:
import os
import shutil
import zipfile
from pathlib import Path

class ZipProcessor:
    def __init__(self, zipname) -> None:
        self.zipname = zipname
        self.temp_directory = Path(f'unzipped-{zipname[:-4]}')
    
    def process_zip(self):
        self.unzip_files()
        self.process_files()
        self.zip_files()

    def unzip_files(self):
        if not os.path.exists(self.temp_directory):
            self.temp_directory.mkdir()
            with zipfile.ZipFile(self.zipname) as zip:
                zip.extractall(str(self.temp_directory))
    
    def zip_files(self):
        with zipfile.ZipFile(self.zipname, 'w') as file:
            for filename in self.temp_directory.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.temp_directory))
    
    def process_files(self):
        pass

將 filename 屬性改為 zipname 屬性避免與各種要繼承的方法中的 filename 搞混  
現在來修正原先的 ZipReplace

In [11]:
class ZipReplace(ZipProcessor):
    def __init__(self, filename, search_string, replace_string) -> None:
        super().__init__(filename)
        self.search_string = search_string
        self.replace_string = replace_string
    
    def process_files(self):
        '''對臨時目錄下的檔案執行搜尋文字與替換'''
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open('w') as file:
                file.write(contents)

In [15]:
!python zipsearch_v2.py hello.zip hello hi

現在我們更容易撰寫其他操作 ZIP 的功能  
試著來做一個改變圖片大小的方法  

In [1]:
! pip install pillow

You should consider upgrading via the 'C:\Users\user\pythonenv\python3_oop\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import sys
from PIL import Image

class ScaleZip(ZipProcessor):
    def process_files(self):
        '''將目錄下的圖檔全部改為 640 X 480'''
        for filename in self.temp_directory.iterdir():
            im = Image.open(str(filename))
            scaled = im.resize((640, 480))
            scaled.save(str(filename))

In [7]:
!python scalezip.py aloha.zip